<a href="https://colab.research.google.com/github/LucasDatilioCarderelli/Codenation/blob/master/Projeto_Final/PrjF_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modeling the Project

## Importings

In [1]:
import pandas as pd
import numpy as np

import matplotlib as plt
import seaborn as sns

from sklearn.preprocessing import (
    LabelEncoder, OneHotEncoder, KBinsDiscretizer)
from sklearn.neighbors import NearestNeighbors

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
mkt = pd.read_csv("/content/mkt")

port1 = pd.read_csv("https://codenation-challenges.s3-us-west-1.amazonaws.com/ml-leads/estaticos_portfolio1.csv")
port2 = pd.read_csv("https://codenation-challenges.s3-us-west-1.amazonaws.com/ml-leads/estaticos_portfolio2.csv")
port3 = pd.read_csv("https://codenation-challenges.s3-us-west-1.amazonaws.com/ml-leads/estaticos_portfolio3.csv")

mkt.shape, port1.shape, port2.shape, port3.shape

((460371, 38), (555, 182), (566, 2), (265, 2))

In [3]:
mkt.head(3)

,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_optante_simples,fl_optante_simei,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,de_nivel_atividade,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,qt_socios_st_regular,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,qt_filiais
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,1,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,INDUSTRIA DA CONSTRUCAO,CONSTRUÇÃO CIVIL,14,10 a 15,0,0,0,0,0,2005,0,1,1,SIM,CONSTRUCAO DE EDIFICIOS,CONSTRUCAO,1,0,RN,VERDE,ACIMA DE 1 ANO,ALTA,LESTE POTIGUAR,NATAL,1,2,2,0,44.0,2,"DE R$ 1.500.000,01 A R$ 4.800.000,00","DE R$ 1.500.000,01 A R$ 4.800.000,00",0
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,1,EMPRESARIO INDIVIDUAL,PI,OUTROS,SERVICOS DE ALOJAMENTO/ALIMENTACAO,SERVIÇO,1,1 a 5,0,0,0,1,0,2017,0,1,1,SIM,ALIMENTACAO,ALOJAMENTO E ALIMENTACAO,0,1,PI,CINZA,ATE 1 ANO,BAIXA,CENTRO NORTE PIAUIENSE,TERESINA,1,1,1,0,27.0,1,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",0
2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,1,EMPRESARIO INDIVIDUAL,AM,OUTROS,"TRANSPORTE, ARMAZENAGEM E CORREIO",SERVIÇO,7,5 a 10,0,0,0,1,0,2011,0,0,1,SIM,TRANSPORTE TERRESTRE,TRANSPORTE ARMAZENAGEM E CORREIO,1,1,AM,AMARELO,ACIMA DE 1 ANO,MEDIA,CENTRO AMAZONENSE,MANAUS,1,1,1,0,32.0,0,"ATE R$ 81.000,00","ATE R$ 81.000,00",0


## Preprocessing

### Dropping

In [4]:
mkt.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460371 entries, 0 to 460370
Data columns (total 38 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   id                                   460371 non-null  object 
 1   fl_matriz                            460371 non-null  int64  
 2   de_natureza_juridica                 460371 non-null  object 
 3   sg_uf                                460371 non-null  object 
 4   natureza_juridica_macro              460371 non-null  object 
 5   de_ramo                              460371 non-null  object 
 6   setor                                460371 non-null  object 
 7   idade_empresa_anos                   460371 non-null  int64  
 8   idade_emp_cat                        460371 non-null  object 
 9   fl_me                                460371 non-null  int64  
 10  fl_sa                                460371 non-null  int64  
 11  fl_epp       

In [5]:
mkt["idade_media_socios"] = mkt["idade_media_socios"].astype("int64")

In [6]:
mkt.select_dtypes(include=object).nunique()

id                                     460371
de_natureza_juridica                       66
sg_uf                                       6
natureza_juridica_macro                     7
de_ramo                                    32
setor                                       5
idade_emp_cat                               6
fl_rm                                       2
nm_divisao                                 87
nm_segmento                                21
sg_uf_matriz                               27
de_saude_tributaria                         6
de_saude_rescencia                          5
de_nivel_atividade                          4
nm_meso_regiao                             19
nm_micro_regiao                            73
de_faixa_faturamento_estimado              12
de_faixa_faturamento_estimado_grupo        11
dtype: int64

In [7]:
try: mkt.drop(["idade_empresa_anos", "nm_meso_regiao"], axis=1, inplace=True) 
except: print("droped")

---

In [8]:
X = mkt.copy()

### Bin continuous data into intervals

In [9]:
X.select_dtypes(include="number")

,fl_matriz,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_optante_simples,fl_optante_simei,fl_passivel_iss,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,qt_socios_st_regular,qt_filiais
0,1,0,0,0,0,0,2005,0,1,1,1,0,1,2,2,0,44,2,0
1,1,0,0,0,1,0,2017,0,1,1,0,1,1,1,1,0,27,1,0
2,1,0,0,0,1,0,2011,0,0,1,1,1,1,1,1,0,32,0,0
3,1,0,0,0,0,0,2012,0,1,1,1,0,1,1,1,0,36,1,0
4,1,0,0,0,0,0,2015,0,1,1,1,0,1,1,1,0,36,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460366,1,0,0,0,1,0,2015,0,1,1,1,1,0,1,1,0,31,1,0
460367,1,0,0,0,0,0,2009,0,0,0,0,0,1,0,0,0,36,0,0
460368,1,0,0,0,0,0,2005,0,0,0,0,0,1,1,1,0,36,1,0
460369,1,0,0,0,0,0,2016,0,1,0,0,0,0,5,5,0,31,3,0


In [10]:
est = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
est.fit(X[["dt_situacao", "idade_media_socios"]])
X[["dt_situacao", "idade_media_socios"]] = est.transform(
    X[["dt_situacao", "idade_media_socios"]]).astype(int)

In [11]:
X["dt_situacao"].unique(), X["idade_media_socios"].unique()

(array([4, 3, 2, 0, 1]), array([2, 0, 1, 3, 4]))

### Enconding

In [12]:
cat_cols = X.select_dtypes(include=object).columns[1:]
cat_cols

Index(['de_natureza_juridica', 'sg_uf', 'natureza_juridica_macro', 'de_ramo',
       'setor', 'idade_emp_cat', 'fl_rm', 'nm_divisao', 'nm_segmento',
       'sg_uf_matriz', 'de_saude_tributaria', 'de_saude_rescencia',
       'de_nivel_atividade', 'nm_micro_regiao',
       'de_faixa_faturamento_estimado', 'de_faixa_faturamento_estimado_grupo'],
      dtype='object')

In [13]:
X[cat_cols] = X[cat_cols].apply(lambda col: LabelEncoder().fit_transform(col))
X

,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_emp_cat,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_optante_simples,fl_optante_simei,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,de_nivel_atividade,nm_micro_regiao,fl_passivel_iss,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,qt_socios_st_regular,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,qt_filiais
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,1,59,4,2,11,2,1,0,0,0,0,0,4,0,1,1,1,32,10,1,0,19,4,0,0,48,1,2,2,0,2,2,2,2,0
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,1,17,3,5,20,4,0,0,0,0,1,0,4,0,1,1,1,3,3,0,1,16,2,1,1,69,1,1,1,0,0,1,10,10,0
2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,1,17,1,5,31,4,3,0,0,0,1,0,4,0,0,1,1,86,20,1,1,2,0,0,2,43,1,1,1,0,1,0,1,1,0
3,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,1,17,1,5,24,4,3,0,0,0,0,0,4,0,1,1,1,74,17,1,0,2,0,0,2,43,1,1,1,0,1,1,10,10,0
4,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,1,17,4,5,26,4,0,0,0,0,0,0,4,0,1,1,1,77,8,1,0,19,4,0,0,48,1,1,1,0,1,0,10,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460366,469ee5fc984d0bb60aa2f40cda4d003d7c9b963d149ed3...,1,17,4,5,3,1,0,0,0,0,1,0,4,0,1,1,0,28,9,1,1,19,4,0,0,14,0,1,1,0,1,1,1,1,0
460367,656f496e7e2fad708f0022d8ad97d2cb9344e839a5f3cb...,1,17,3,5,19,4,3,0,0,0,0,0,4,0,0,0,1,76,5,0,0,16,3,0,3,69,1,0,0,0,1,0,10,10,0
460368,9b933a5cf2149d1338f00fb459d34a16b9b0f2e8d3e8e1...,1,0,2,3,23,4,5,0,0,0,0,0,4,0,0,0,1,11,18,0,0,9,0,0,2,0,1,1,1,0,1,1,10,10,0
460369,84db0f17467cf4f02439e01ade57150b6ac59d9dae53ed...,1,59,5,2,5,1,0,0,0,0,0,0,4,0,1,0,0,30,9,0,0,20,4,0,0,43,0,5,5,0,1,3,7,7,0


### Preparing the Portifolios

In [14]:
port1 = port1[["id"]]

In [15]:
port2.drop("Unnamed: 0", axis=1, inplace=True)
port3.drop("Unnamed: 0", axis=1, inplace=True)

In [16]:
port1 = port1.merge(right=X, how="inner", on="id")
port2 = port2.merge(right=X, how="inner", on="id")
port3 = port3.merge(right=X, how="inner", on="id")

In [17]:
ports = pd.concat([port1, port2, port3])
ports.index

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            255, 256, 257, 258, 259, 260, 261, 262, 263, 264],
           dtype='int64', length=1386)

---

In [18]:
try:
  X.set_index('id', inplace=True)
  port1.set_index('id', inplace=True)
  port2.set_index('id', inplace=True)
  port3.set_index('id', inplace=True)
except: print("seted")

In [19]:
X = X.drop(index=ports["id"])

In [20]:
X.shape

(459093, 35)

## Fitting

In [22]:
%%time
nn = NearestNeighbors(n_neighbors=1)
nn.fit(X)

CPU times: user 17min 14s, sys: 3.1 s, total: 17min 17s
Wall time: 17min 16s


### Getting the Leads

In [23]:
distances, indices = nn.kneighbors(port1)

In [24]:
port1_leads = pd.DataFrame([X.iloc[indices[row,col]] for row in range(indices.shape[0]) for col in range(indices.shape[1])])

In [25]:
port1_leads

,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_emp_cat,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_optante_simples,fl_optante_simei,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,de_nivel_atividade,nm_micro_regiao,fl_passivel_iss,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,qt_socios_st_regular,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,qt_filiais
261dd4fcf6e94e567442398642ab29e4c36baa128c9983ad099b99c21eea1530,1,17,3,5,5,1,4,0,0,0,1,0,4,0,0,1,0,30,9,0,1,16,2,4,1,61,0,1,1,0,3,1,10,10,0
ef960aff7dcb3407a0567e98034b87891b0c6e361d6cc42acf218c2ee67080f9,0,17,2,5,5,1,3,0,0,0,0,0,4,0,0,1,0,30,9,0,0,9,4,0,2,44,0,0,0,0,1,0,10,7,2
d1ea8ea605398098a93bcf32e1d3bde38dd25fd6488f9cafccaa8a5d0e58d29f,1,17,2,5,5,1,3,0,0,0,1,0,4,0,0,0,1,30,9,1,1,9,1,0,2,0,0,1,1,0,3,1,1,1,0
45bef5a6ab38afe6b353cb7be0f4ecc9037a8f993913bba33023f4e91cd14178,1,17,4,5,5,1,5,0,0,0,0,0,4,0,0,0,0,30,9,1,1,19,4,0,0,70,0,1,1,0,1,0,1,1,0
37804b2f7c3cbba38d5ba55e85c657f389dbfa2d0cec661a38b898b457a082ba,1,59,4,2,5,1,1,0,0,0,0,0,4,0,1,1,0,30,9,1,0,19,4,0,0,47,0,2,2,0,1,2,2,8,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ab7b39c87e98275d546ba553da2080111c4b222375a1db63263483e2fdda8bbe,1,59,2,2,2,3,5,0,0,0,0,0,4,0,0,0,1,31,13,0,0,9,3,0,2,0,0,0,0,0,1,0,10,10,0
f6808901a1cf975244fdabbfa8dcd073286fd9673ae3acb38456e9bfd5966827,1,17,2,5,5,1,0,0,0,0,1,0,4,0,1,1,0,30,9,0,1,9,2,1,1,20,0,1,1,0,2,1,10,10,0
098a4f3772359778c30dbb4486715af54b21cf90fd288e39dfc7f5fae8a1a301,1,17,3,5,5,1,5,0,0,0,0,0,4,0,0,0,1,30,9,0,0,16,1,0,2,69,0,0,0,0,1,0,10,10,0
4b80cb9dcbd86aa492d2c69a973f2c1d24d9def1528f72c5adf4af28c452cb17,1,17,2,5,5,1,4,0,0,0,1,0,4,0,1,1,0,30,9,0,1,9,2,1,1,52,1,1,1,0,1,1,10,10,0


In [26]:
%%time
distances, indices = nn.kneighbors(port2)
port2_leads = pd.DataFrame([X.iloc[indices[row,col]] for row in range(indices.shape[0]) for col in range(indices.shape[1])])
distances, indices = nn.kneighbors(port3)
port3_leads = pd.DataFrame([X.iloc[indices[row,col]] for row in range(indices.shape[0]) for col in range(indices.shape[1])])

CPU times: user 1.29 s, sys: 3 ms, total: 1.29 s
Wall time: 1.3 s


In [27]:
port2_leads

,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_emp_cat,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_optante_simples,fl_optante_simei,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,de_nivel_atividade,nm_micro_regiao,fl_passivel_iss,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,qt_socios_st_regular,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,qt_filiais
891e3ea062d9f034849e7fc41fd91603daa949cdca184f11adbbb8ca80f45f8e,1,2,0,0,0,4,3,0,0,0,0,0,4,0,1,1,0,0,0,0,0,0,4,0,0,57,1,0,0,0,1,0,7,7,0
5317b65902700f9566ceb9e4b4f5ed1fa0098f805052f6cbd53bd35dd52698a0,1,33,4,5,0,4,5,0,0,0,0,0,4,0,0,0,0,0,0,0,0,19,4,0,0,1,1,0,0,0,1,0,4,4,0
0526b8c9dc45364b164d7138e6a2a0d61393925951703ebd1b03a93f1be8c48f,0,63,4,2,21,4,3,0,0,0,0,1,4,0,1,0,0,36,11,1,0,19,4,0,0,14,1,2,2,0,1,2,7,2,3
31a9b774217cdb320bc2881c917729bb58ce9722958854e76659ba387c69d61f,1,33,1,5,0,4,5,0,0,0,0,0,4,0,0,0,0,0,0,0,0,2,1,0,0,6,1,0,0,0,1,0,4,4,0
7d96561f96275a0290c8edd1d3bc0cc25fafc4c779a65b000debe20ea1052f03,1,59,4,2,16,0,5,0,0,0,0,0,4,0,1,1,1,2,1,0,0,19,4,0,0,48,0,4,4,0,4,4,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
f5d23b002d394b8d89c7ba37f5c9cae5926f897e8e781ffd97a10c5601ee6a3c,1,59,2,2,12,4,2,0,0,0,0,0,4,0,1,1,1,15,15,1,0,9,1,0,0,0,1,4,4,0,3,4,7,7,0
6bcf8b3f78f6399f66cd2ccb777162d8f7138681987d1003be56d1c3377691d9,1,59,1,2,4,1,2,0,0,0,0,0,4,0,1,1,1,29,9,0,0,2,4,0,0,43,0,2,2,0,3,2,5,5,2
f295a18ca8c1150426ba89c101679b909b1b99a2e1a86997e6f2d3b7061b9067,1,59,4,2,2,3,5,0,0,0,0,0,4,0,1,0,1,51,13,1,0,19,1,0,0,48,0,2,2,0,3,2,2,2,0
c9f3ed8fd2cb9721896e6f75e3cdd413c923103c966529d9c06f09f8e1008bb3,0,51,5,3,21,4,2,0,0,0,0,0,4,0,1,1,0,36,11,0,0,20,4,0,0,43,1,0,0,0,1,0,3,4,10


In [28]:
port3_leads

,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_emp_cat,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_optante_simples,fl_optante_simei,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,de_nivel_atividade,nm_micro_regiao,fl_passivel_iss,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,qt_socios_st_regular,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,qt_filiais
81e062f5554447eb5a8ddfdcfe63ce42752b4cb311f1e84644bb35a856465ab9,1,33,2,5,0,4,5,0,0,0,0,0,4,0,0,0,0,0,0,0,0,9,4,0,0,52,1,0,0,0,1,0,5,5,0
0526b8c9dc45364b164d7138e6a2a0d61393925951703ebd1b03a93f1be8c48f,0,63,4,2,21,4,3,0,0,0,0,1,4,0,1,0,0,36,11,1,0,19,4,0,0,14,1,2,2,0,1,2,7,2,3
70897f415c4473d8ca650d0ff4a567c5bb0f7a87df2f320d442628638bfe8a59,1,41,1,0,0,4,5,0,0,0,0,0,4,0,0,0,1,0,0,0,0,2,1,0,0,43,1,0,0,0,1,0,4,4,0
dcdabcef5d815e5e6e5bab3a553811d1ad9e628d61e91d3cbbbf0f95afe7ad1b,1,33,3,5,0,4,5,0,0,0,0,0,4,0,0,0,0,0,0,0,0,16,4,0,0,72,1,0,0,0,1,0,3,3,0
5b98116ad29f54d3f6260f72752f010ad61b526c1e1222c523eef90e09536b89,1,33,4,5,0,4,5,0,0,0,0,0,4,0,0,0,0,0,0,0,0,19,1,0,0,7,1,0,0,0,1,0,5,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95461a3b1bab5d6758787474cb498c19a695a0fde0b987d6c197db7aecb6988b,1,13,5,5,26,4,1,0,0,0,0,0,4,0,1,1,0,17,8,1,0,20,4,0,0,43,1,1,1,0,1,1,1,1,0
a8e71c34b68ec0b98d73e21a91c7dbb8e072e9837643309b5aefd7e1407f0283,0,59,3,2,23,4,3,0,0,0,0,0,4,0,0,1,1,11,18,0,0,16,4,0,0,69,1,4,4,0,1,3,2,5,3
c0fc190e4395616c2683cacfa14215f42304a0d299eae8ecc6ef46bfeef63d58,1,59,1,2,26,4,2,0,0,0,0,0,4,0,1,1,1,77,8,0,0,2,4,0,0,43,1,2,2,0,3,2,7,7,0
5c012008ad74c1eea3433ec7e5fa5584dafd54a7c9cdab01f68d4ad0ec0b52b4,1,12,0,2,18,4,2,0,0,0,0,0,4,0,0,1,0,75,6,0,0,0,1,0,0,57,1,3,3,0,3,3,7,7,0


In [29]:
# Seeing if there is equals ids
cont = 0
for row1 in port3.index:
  for row2 in port3_leads.index:
    if row1 == row2:
      cont+=1
print(cont)

0
